In [ ]:
import importlib
import arbitragem
importlib.reload(arbitragem)
import itertools
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit

## Simulação

In [29]:
data_inicio = '2024-06-20'
data_fim   = '2025-07-19'

janelas= [500,1000,1500,2000,2500,3000,3500,4000,45000]

retorno_pct, retorno_risco, sharpe = arbitragem.simular_estrategia(
    moeda1='SHIB',
    moeda2='BNB',
    zscore_compra_e_venda=2.0,
    zscore_encerrar_posicao=0.5,
    stop_loss=3.0,
    cooldown_stop_loss=20,
    janela=11,
    data_inicial='2025-04-20',
    data_final='2025-07-19',
    periodo_observacoes="1d",
    )
print(f"Simulação Janela: {i}")
print(f"Retorno percentual: {retorno_pct*100:.2f}%")
print(f"Retorno ajustado ao risco: {retorno_risco:.4f}")
print(f"Índice de Sharpe: {sharpe:.4f}")





[False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, True, True, False]
Retorno total: -1.02%


NameError: name 'i' is not defined

## Função

In [37]:
def validacao_cruzada_temporal(df, moeda1, moeda2, param_grid, n_splits=4, taxa=0.001):
    """
    Validação cruzada temporal para otimizar zscore_compra_e_venda, 
    zscore_encerrar_posicao e número de janelas.

    Parâmetros:
    - df: DataFrame com preços dos dois ativos (Ativo1 e Ativo2)
    - moeda1, moeda2: nomes dos ativos
    - param_grid: dicionário de parâmetros a otimizar
    - n_splits: número de divisões temporais
    - taxa: taxa de operação
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    melhores_parametros = None
    melhor_score = -np.inf
    resultados = []

    for params in ParameterGrid(param_grid):
        scores = []

        for train_idx, val_idx in tscv.split(df):
            df_train = df.iloc[train_idx]
            df_val = df.iloc[val_idx]

            # Evita splits com menos de 10 linhas
            if len(df_train) < 10 or len(df_val) < 10:
                continue

            try:
                retorno_pct, _, _ = arbitragem.simular_estrategia(
                    moeda1=moeda1,
                    moeda2=moeda2,
                    zscore_compra_e_venda=params['zscore_compra_e_venda'],
                    zscore_encerrar_posicao=params['zscore_encerrar_posicao'],
                    stop_loss=0.0,
                    cooldown_stop_loss=0,
                    janela=params['janela'],
                    data_inicial=df_train.index[0].strftime('%Y-%m-%d'),
                    data_final=df_val.index[-1].strftime('%Y-%m-%d'),
                    periodo_observacoes="1d",
                    taxa=taxa,
                    capital_inicial=10000
                )

                if retorno_pct is not None:
                    scores.append(retorno_pct)
            except Exception as e:
                # Ignora erros e continua
                continue

        media_retorno = np.mean(scores) if scores else -np.inf
        resultados.append((params, media_retorno))

        if media_retorno > melhor_score:
            melhor_score = media_retorno
            melhores_parametros = params

    df_resultados = pd.DataFrame(resultados, columns=["params", "retorno_medio"])
    return melhores_parametros, df_resultados

# Simulação multipla

In [43]:
moedas = ['BTC', 'BNB', 'ETH', 'SOL', 'ADA', 'XRP', 'DOGE', 'LTC', 'TRX', 'DOT', 'SHIB', 'BCH', 'TON', 'DAI', 'AVAX']
m1 ='SHIB'
m2 = 'ETH'
periodo_observacoes = '1d'

# Carregar dados
df_total = arbitragem.carregar_dados(m1, m2, periodo_observacoes).loc['2025-01-01':'2025-04-19']

param_grid = {
    'zscore_compra_e_venda': [1.5, 2.0, 2.5],
    'zscore_encerrar_posicao': [0, 0.2, 0.3, 0.5],
    'janela': [5, 10, 15, 20]
}

melhores_parametros, df_resultados = validacao_cruzada_temporal(df_total, m1, m2, param_grid, n_splits=4)
print("Melhores parâmetros:", melhores_parametros)
df_resultados.sort_values("retorno_medio", ascending=False).\
    to_excel(f"resultados/otimizacao/{periodo_observacoes}/{m1}-{m2}- Resultado Validacao Cruzada.xlsx")


[False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True]
Retorno total: 0.00%
[False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]
Retorno total: 0.00%
[False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,

In [ ]:
# Validação final
retorno_val, risco_val, sharpe_val = arbitragem.simular_estrategia(
    moeda1=m1,
    moeda2=m2,
    zscore_compra_e_venda=melhores_parametros['zscore_compra_venda'],
    zscore_encerrar_posicao=melhores_parametros['zscore_encerrar_posicao'],
    stop_loss=0.0,
    cooldown_stop_loss=0,
    janela=melhores_parametros['janela'],
    data_inicial='2025-05-20',
    data_final='2025-07-19',
    periodo_observacoes=intervalo,
    taxa=0.01,
    capital_inicial=10000
)

print(f"\nValidação final:")
print(f"Retorno: {retorno_val*100:.2f}% | Sharpe: {sharpe_val:.2f}")

In [ ]:
df = pd.read_csv('pares.cointegrados.csv') #todos os pares

df_cointegrado = df[df['P-Valor'] < 0.05]

pares_cointegrados = [
    ["BTC","BNB"],
    ["BNB", "XRP"],
]

for cada_par in pares_cointegrados:
    #RODAR SIMULACAO
    print(cada_par)

['BTC', 'BNB']
['BNB', 'XRP']


## Teste Manual

In [ ]:

# Algoritmo de otimizacao
zscv = 1
zse = 0.5

stop_loss = 0

cooldown = 0
janela = 60

dfcodigo = carregar_dados("ETH", "WBTC")

# Intervalo para teste de cointegração
data_inicial = '2025-06-20'
data_final   = '2025-07-19'
df_filtrado = dfcodigo.loc[data_inicial:data_final]


df_sinalizado = gerar_sinais(df_filtrado, zscv, zse, stop_loss, cooldown, janela) #gera os sinais de compra_venda_encerramento

capital_final, retorno_percentual, capital_series, df_com_retorno_financeiro = simular_retorno_por_trade(df_sinalizado) #simula os retornos

#Valores pra gráfico
df_com_retorno_financeiro['ativo1_valor_esperado'] = df_com_retorno_financeiro['Ativo2'] + df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo1_desvio_preco'] = df_com_retorno_financeiro['ativo1_valor_esperado'] - df_com_retorno_financeiro['Ativo1']
df_com_retorno_financeiro['ativo1_std_preco'] = df_com_retorno_financeiro['ativo1_desvio_preco'] / df_com_retorno_financeiro['rolling_std']

df_com_retorno_financeiro['ativo2_valor_esperado'] = df_com_retorno_financeiro['Ativo1'] - df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo2_desvio_preco'] = df_com_retorno_financeiro['ativo2_valor_esperado'] - df_com_retorno_financeiro['Ativo2']
df_com_retorno_financeiro['ativo2_std_preco'] = df_com_retorno_financeiro['ativo2_desvio_preco'] / df_com_retorno_financeiro['rolling_std']


df_com_retorno_financeiro.to_excel('retornos.xlsx', index=False)

print(f"\nRetorno acumulado: {retorno_percentual * 100:.2f}%")
print(f"Capital final: ${capital_final:.2f}")
